## Reinforcement Learning

Рекомендованная литература:

* Barto Sutton Reinforcement Learning: An Introduction (классика теории)
* Lapan Deep Reinforcement Learning Hands-On Second Edition (Книга с большим количеством примеров по RL)
* Sudharsan Ravichandiran Hands-On Reinforcement Learning with Python (Тоже много примеров)

Прочая дополнительная литература:

* Alex Zai and Brandon Brown Deep Reinforcement Learning in Action


Полезные ссылки:
* https://www.alexirpan.com/2018/02/14/rl-hard.html Deep Reinforcement Learning Doesn't Work Yet
* https://openai.com/blog/faulty-reward-functions/  Faulty Reward Functions in the Wild

In [ ]:
import sys, os

################################################
# For MDP
################################################

print(os.getcwd())

if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week02_value_based/mdp.py
    !touch .setup_complete
    
else:
    sys.path.append("../../src/L15_RL/lib")
    print('Not Colab')


# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

if 'google.colab' in sys.modules:
    !rm .setup_complete
    

################################################
# For CartPole
################################################
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
        
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/atari_wrappers.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/utils.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/replay_buffer.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week04_approx_rl/framebuffer.py

    !pip install gym[box2d]

    !touch .setup_complete
else:
    sys.path.append("../../src/L15_RL/lib")
    print('Not Colab')  
    

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'


import random
import numpy as np
import torch
# ../../src/L15_RL/lib/

# import src.L15_RL.lib.utils
import utils

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn

## Intro



### Обучение с учителем
Если у нас есть набор примеров с правильными ответами, то мы используем эту выборку для обучения нашей модели, а после обучения, применяем её к неразмеченным данным. Именно этот подход мы использовали, когда обучали классификатор для MNIST, подавая на вход сети картинки с изображениями рукописных цифр и считая градиент для подстройки весов на основе разницы между известным лэйблом цифры и выходом нейросети.
### Обучение без учителя
В некоторых случаях у нас нет размеченных данных, на которых мы могли бы заранее обучить модель. Но, при решении некоторых задач, можно обойтись без размеченной выборки. Примером такой задачи является задача кластеризации.
### Обучение с подкреплением
В некоторых случаях существующие методы обучения без учителя нам не подходят. В то же время у нас нет возможности создать качественную обучающую выборку. При этом мы можем постфактум оценить действия нашей модели и использовать эту оценку подстроить модель так, чтобы она чаще совершала желательные действия и реже - нежелательные. В литературе такую оценку называют вознаграждением (reward), а обучение строится таким образом, чтобы это модель стремилась максимизировать получаемое вознаграждение.

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-1-7.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-1-17.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

## Терминология: агент, функция награды, состояние среды

Агент и среда - ключевые понятия в обучении с подкрелением.

**Агент** - программа, принимающая решение о дальнейших действиях на основе информации о состоянии среды.

**Среда** - это мир, в котором агент должен "выживать", т.е. всё, с чем агент может прямо или косвенно взаимодействовать. Среда обладает состоянием (State), агент может влиять на среду, совершая какие-то действия (Actions), переводя среду при этом из одного состояния в другое и получая какое-то вознаграждение. Среда описывается пространством возможных состояний. Конкретное состояние - вектор в этом пространстве.

В зависимости от конкретной задачи, агент может наблюдать либо полное состояние среды, либо только некоторую его часть. Во втором случае, агенту может потребоваться какое-то внутреннее представление полного состояния, которое будет обновляться по мере получения новых данных.

**Фукнция награды** - вводимая программистом формула вычисления ценности действия на основе финального результата, ожидания этого результата, промежуточных результатов и любых других параметров, которые будут подсказывать путь к наилучшей последовательности действий агента. Это некоторый аналог функции потерь, без которой непонятно чему учиться. Например, в шахматах истинная награда это победа, но взятая фигура соперника тоже ценна и должна увеличивать награду, если мы хотим подсказать агенту, что брать чужие фигуры полезно. Может ли после этого агент получать мат, позарившись на незащищенную фигуру? Да, ровно как и неопытный шахматный игрок. Попытка передать через дополнительные неосновные награды подсказки к получению основной награды называется **reward shaping**.

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-1-8.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-1-9.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


Этот список ожно дополнить несколькими примерамии:
* drug discovery: обычно функции отражающие полезность лекарства недифференицруемы и можно искать оптимальные, вводя RL
* не так давно одна российская металлургическая компания использовала RL для оптимаизации работы прокатного стана
* NAS (Neural Architecture Search): при поиске оптимальных топологий сети можно решать задачу с помощью RL, награждая нашего агента за нахождение ххороших топологий и наказывая за плохие

### MDP

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-24_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-4.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-3.png" alt="Drawing" style="width: 500px;"/> </td>
    <td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-25_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-12.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-13.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


Введние discounting необходимо по целому ряду причин. Одной из основных причин явялется необходимость ограничения максимальной суммы наград (для избегания бесконечных циклов вознаграждения). В самом деле:

Пусть $R_{max} = max R_i$ - максимальная награда
$G_t = R_{t+1} + \gamma * R_{t+2} + \gamma^2 * R_{t+3} + ... \le R_{max} + \gamma * R_{max} + \gamma^2 * R_{max} + ... = R_{max} (1 + \gamma + \gamma^2 + ... ) \le R_{max} * \frac{1}{1-\gamma} <= const $

Также, это отражает поведения животных и человека, который больше предпочитают моментальную награду, чем отложенную на будущее.


<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-15.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-17.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-26.png" alt="Drawing" width="500">
</tr></table>

Стоит отметить, что политика, как следует из определения не обязательно должна быть жестко зафиксированной. Вполне возможно, что она будет случайной. Это может, к примеру, пригодиться при исследованиях среды.

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-28.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-29_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Зная теперь понятия MDP и политики нам хочется находить такую политику, которая будет максимизировать наши потенциальные выигрыши $G_t$.


Для этого нам понадобятся  две важнейшие в RL функции: V и Q. 

* $V_{\pi}(s)$ является матожиданием дисконтрованных наград, которые мы получим если будем из состояния s действовать согласно политике $\pi$

* $Q_{\pi}(s, a)$ является матожиданием дисконтрованных наград, которые мы получим если сделаем из состояния s действие a и **дальше** будем действовать согласно политике $\pi$

То есть в q мы в отличии от v фиксируем наше первое действие (не обязательно соответсвующее политике), а в v мы первое действие выбираем согалсно политике $\pi$

Под $E_\pi$ мы подразумеваем стохастичность и политки и среды

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-30.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-31.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

* Вместо $q_\pi(S_{t+1}, A_{t+1})$ должна быть $v_\pi(S_{t+1})$

Q функция убирает стохастичность политики на первом шаге. С помощью нее мы фиксируем наш первый шаг из этого состояния.

На правом слайде мы расписываем v как сумму произведений вероятностей действий (это определяет политика) и q функций 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-33_.png" alt="Drawing" style="width: 500px;"/> </td>
    <td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-35__.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Распишем $E_\pi$ глубже. Здесь еще добавляется в явном виде случайность среды, которая в зависимости от текущего s и выбранного a, выдает нам новый s' и r. На диаграмме получается, что верхние ребра дерева символизируют случайность политики, а нижние - случайность среды


На правом слайде рассчитаны значени v функции для случайной политик (каждое действие в каждом состоянии равновероятно)

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-32_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-34_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Похожее раскрытие матожидания можно проделать и для Q, но в нем сначала будет идти случайность среды

<img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-36.png" alt="Drawing" width="500">

Если все записать в матричной форме, то можно увидеть, что Bellman Expectaion Equation имеет решение (что нельзя сказать о поиске оптимальной политике, как мы увидим ниже)

### Давайте теперь искать наилучшую политику или наилучший способ поведения

Умея сравнивать ожидания от политик в кажом состоянии не сложно начать сравнивать политики целиком 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-37.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-38_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Для начала определим оптимальные функции v и q

Но знание об оптимальных q эквивалентно знанию об оптимальном действии (А почему?)

На графе MDP мы расситали оптимальные значения V (алгоритм их получения вы узнаете чуть позже)

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-39_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-40.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

В примере подсчитаны q* для кажого конкретного действия и приписаны к ребрам, отвечающим за эти действия

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-41.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-42_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-43.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-45_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Зная оптималную оценку мы должны действовать жадно, если хотим ее достичь. В самом деле, любое другое действие снизит наш выигрыш

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-44_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-46_.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

* Эти условия и есть уравнения оптимальности Белммана. Их можно интерпретировать следующим образом: если мы точно знаем оптимальные q функции в будущем (то есть, знаем сколько мы получим наград, если из будущих состояний будем действовать определенным способом) и знаем моментальные награды, то из всех доступных нам сейчас действия нужно выбрать, то которое максимизирует сумму нашей немедленной награды и будущих наград при условии оптимального поведения в будущем. ~ **На каждом шаге следует выбирать оптимальное решение в предположении об оптимальности всех последущих шагов**.

* Но какое поведение на следюущем шаге при известных Q функциях может быть оптимальным? Очевидно, то которое максимизирует Q.

* А принцип оптимальности Беллмана говорит следующее: оптимальная стратегия имеет свойство, что какими бы ни были начальное состояние и начальное решение, последующие решения должны составлять оптимальный курс действий по отношению к состоянию, полученному в результате первого решения ~ **оптимальная стратегия зависит только от  текущего состояния и цели и не зависит от предыстории**.

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-47_.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-48.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-2-49.png" alt="Drawing" width="500">

## Bellman

### Policy evaluation

Если на картинках, привденных выше с игрушечным MDP попробовать самому просчитать $v_\pi$, то можно заметить, что вы со временем приедете к рекурсии и не очень понятно что с этим делать. В теории это можно превратить в СЛАУ и решить, но это нам не очень подойдет. 

Тут можно воспользоваться идеями динамического программирования, так как благодаря уравнениям Беллмана у нас есть рекурсивное разложение => оптимальная подструктура. А также можно переиспользовать полученные ранее оценки v

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-56.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-67.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


Идея bellman evaluation заключается в том, чтобы много раз применять bellman expectation equation. И можно доказать, что этот процесс сойдется в итоге к точной оценке значений $V_\pi (s)$ для всех s

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-3-7.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-3-9.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-3-10.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-3-11.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


На слайдах выше слева представлены, получаемые итеративно оценки $V_\pi$, а справа жадная плоитика на основе этих оценок. Как мы видим на слайдах справа, со временем мы сходимся к оптимальной политике (почему так происходит подробнее объясняется ниже). Интуитивно это объясняется так: жадная политика гарантированно будет только улучшением по сравнению со случайной политикой, и в конце концов это нас приведет к оптимальной политике

### Policy improvement

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-60.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/silver-3-13.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


Можно доказать, что такой жадный выбор не ухудшает, а возможно, улучшает политику. Это policy improvement theorem (док-во смотри в BurtoSutton стр 78)

Идея Generalized policy iteration
* Когда мы выполнили процедуру оценки - наша политика перестала быть жадной относительно новой v функции. 
* И наоборот, когда мы выбралии жадные действия - наша оценка v перестала ей соответсвовать

Выполняя этот цикл гаша политика сходится к оптимальной

Также не имеет значения с какой политики вы стартовали, все равно данный метод сойдется. (Это следует из теоремы Банаха)

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-72.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-75.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

### Value iteration

Одним из недостатков policy iteration заключается в том, что каждая из ее итераций включает в себя оценку политики, которая сама по себе может быть длительным итеративным вычислением, требующим многократных проходов по набору состояний. Если оценка политики выполняется итеративно, то сходимость точно к $V_\pi$ происходит только в пределе. Должны ли мы ждать точного схождения, или мы можем не останавливаться на достигнутом? Пример с жадным выбором политик выше показывает, что можно усечь этап policy evaluation. В этом примере итерации policy evaluation за пределами первых трех не влияют на соответствующую жадную политику.

Фактически, этап policy evaluation of policy iteration  может быть сокращен различными способами без потери гарантий сходимости policy iteration. Один важный особый случай - это когда оценка политики останавливается сразу после одного цикла (одно обновление каждого состояния). Этот алгоритм называется value iteration. Его можно записать как  простую операцию обновления, которая сочетает в себе этапы улучшения политики и усеченной policy evaluation (один шаг).

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-77.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-2-78.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


#### Отличие value iteration и policy iteration
Де факто value iteration это policy iteration с одним циклом policy evaluation.

## Библиотека OpenAI Gym

В OpenAI создали готовую библиотеку для моделировавния, визуализирования и обучения, что не только удобно для тестов. Одно из хороших преимуществ это качественный абстрактный интерфейс взаимодействия среды и агента, который позволяет делать совместимые с Gym среды и впоследствии интегрироваться в экосистему, легко подменять типы агентов, тестировать, сравнивать.
Среди готовых сред для взаимодействия с ними есть
- игры Atari - полноценные компьютерные игры, типа Арканоида, где вместо человека в компьютер играет агент.
- классические настольные пошаговые игры: шахматы, go.
- физические симуляции, где нужно управлять физическим объектом: маятник, который должен перевернуться и балансировать за счёт раскачивания; тележка, которая должна проехать через холм за счёт раскачивания.
- и т.д.### State-value function

### Пример окружения Gym
В этом примере загружается готовая среда: машинка должна заехать на горку. Функция награды встроенная. А агент - случайное воздействие. Оно ничему не учится, лишь хаотически выдаёт действия, но это показывает где взять все необходимые данные для обучения.

In [ ]:
import gym
env = gym.make("MountainCar-v0")
MAX_NUM_EPISODES = 5

for episode in range(MAX_NUM_EPISODES):
    done = False
    obs = env.reset()
    total_reward = 0.0 # To keep track of the total reward obtained in each episode
    step = 0
    while not done:
        env.render()
        
        
        
        action = env.action_space.sample()  # Sample random action.
                                            # This will be replaced
                                            # by our agent's action
                                            # when we # start
                                            # developing the agent algorithms
        
        
        next_state, reward, done, info = \
        env.step(action)  # Send the action to the
                          # environment and receive       
                          # the next_state, reward and
                          # whether done or not
        total_reward += reward
        step += 1
        obs = next_state
    print("\n Episode #{} ended in {} steps. total_reward={}".format(episode, step+1,
total_reward))
env.close()

## MDP practice

<img src="https://upload.wikimedia.org/wikipedia/commons/a/ad/Markov_Decision_Process.svg" width="400px" alt="Diagram by Waldoalvarez via Wikimedia Commons, CC BY-SA 4.0"/>

In [ ]:
transition_probs = {
    's0': {
        'a0': {'s0': 0.5, 's2': 0.5},
        'a1': {'s2': 1}
    },
    's1': {
        'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
        'a1': {'s1': 0.95, 's2': 0.05}
    },
    's2': {
        'a0': {'s0': 0.4, 's2': 0.6},
        'a1': {'s0': 0.3, 's1': 0.3, 's2': 0.4}
    }
}
rewards = {
    's1': {'a0': {'s0': +5}},
    's2': {'a1': {'s0': -1}}
}

from mdp import MDP
mdp = MDP(transition_probs, rewards, initial_state='s0')

# We can now use MDP just as any other gym environment:
print('initial state =', mdp.reset())
next_state, reward, done, info = mdp.step('a1')
print('next_state = %s, reward = %s, done = %s' % (next_state, reward, done))

In [ ]:
# MDP methods

print("mdp.get_all_states =", mdp.get_all_states())
print("mdp.get_possible_actions('s1') = ", mdp.get_possible_actions('s1'))
print("mdp.get_next_states('s1', 'a0') = ", mdp.get_next_states('s1', 'a0'))

# state, action, next_state
print("mdp.get_reward('s1', 'a0', 's0') = ", mdp.get_reward('s1', 'a0', 's0'))

# get_transition_prob(self, state, action, next_state)
print("mdp.get_transition_prob('s1', 'a0', 's0') = ", mdp.get_transition_prob('s1', 'a0', 's0'))

### Value iteration

Давайте теперь попробуем решить задачу максимизации на этом MDP с помощью __V__alue __I__teration__

Псевдокод VI:

---

`1.` Initialize $V^{(0)}(s)=0$, for all $s$

`2.` For $i=0, 1, 2, \dots$
 
`3.` $ \quad V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$, for all $s$

---


Для начала, реализуем функцию для вычисление state-action value function $Q^{\pi}$, которая определяется следующей формулой: 

$$Q_i(s, a) = \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$$

In [ ]:
# Value iteration

def get_action_value(mdp, state_values, state, action, gamma):
    """ Computes Q(s,a) as in formula above """    
    next_states = mdp.get_next_states(state, action)
    
    Q = 0.0
    
    for next_state in next_states:
        p = mdp.get_transition_prob(state, action, next_state)
        # state, action, next_state
        Q += p * (mdp.get_reward(state, action, next_state) + gamma * state_values[next_state])
        
    return Q

С помощью $Q(s,a)$ мы можем теперь определить уравнение для получения "следующего" V(s) в value iteration.
 $$V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')] = \max_a Q_i(s,a)$$

In [ ]:
def get_new_state_value(mdp, state_values, state, gamma):
    """ Computes next V(s) as in formula above. Please do not change state_values in process. """
    if mdp.is_terminal(state):
        return 0
    
    actions = mdp.get_possible_actions(state)
    
    Qs = [get_action_value(mdp, state_values, state, action, gamma) for action in actions]
    
    return max(Qs)

Наконец, объединим все в единый работающий алгоритм value iteration.

In [ ]:
# parameters
gamma = 0.9            # discount for MDP
num_iter = 100         # maximum iterations, excluding initialization
# stop VI if new values are this close to old values (or closer)
min_difference = 0.001

# initialize V(s)
state_values = {s: 0 for s in mdp.get_all_states()}

for i in range(num_iter):

    # Compute new state values using the functions you defined above.
    # It must be a dict {state : float V_new(state)}
    # new_state_values = <YOUR CODE>
    new_state_values = {state: get_new_state_value(mdp, state_values, state, gamma) for state in mdp.get_all_states()}

    assert isinstance(new_state_values, dict)

    # Compute difference
    diff = max(abs(new_state_values[s] - state_values[s])
               for s in mdp.get_all_states())
    print("iter %4i   |   diff: %6.5f   |   " % (i, diff), end="")
    print('   '.join("V(%s) = %.3f" % (s, v) for s, v in state_values.items()))
    state_values = new_state_values

    if diff < min_difference:
        print("Terminated")
        break

Теперь, давайте использовать $V^{*}(s)$ для получения оптимальных действий (optimal actions) для всех состояний:

 $$\pi^*(s) = argmax_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')] = argmax_a Q_i(s,a)$$
 
Единственное отличие от поиска V (s) заключается в том, что здесь мы берем не max, а argmax для того, чтобы найти действие с максимальным Q (s, a).

In [ ]:
def get_optimal_action(mdp, state_values, state, gamma=0.9):
    """ Finds optimal action using formula above. """
    if mdp.is_terminal(state):
        return None
    
    actions = mdp.get_possible_actions(state)
    Qs = {action: get_action_value(mdp, state_values, state, action, gamma) for action in actions}
    
    return max(Qs, key=Qs.get)

In [ ]:
# Measure agent's average reward

s = mdp.reset()
rewards = []
for _ in range(10000):
    s, r, done, _ = mdp.step(get_optimal_action(mdp, state_values, s, gamma))
    rewards.append(r)

print("average reward: ", np.mean(rewards))

assert(0.40 < np.mean(rewards) < 0.55)

## HW 1:
Протестировать value iteration на Frozen Lake. Сделать policy iteration.

## Temporal-Difference Learning

### Q-learning

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-14.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-16.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Так-как p(s',r|s,a) нам не дали, то остается только что-то аппроксимировать. Саму p аппроксимировать дорого, поэтому лучше q 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-19.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-20.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Напоминание $G_t = R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + ..$

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-24.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-25.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-27.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-28.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Пусть у нас есть оценка для текущего состояния $S_t$ и действия $A_t$ $Q(S_t, A_t)$.
Сделаем один шаг, получим реальную награду r, посмотрим оценку на следующем шаге $maxQ(S_{t+1}, A)$.
Но в идеале $r + maxQ(S_t, A) = Q(S_t, A_t)$, но если есть разница, то ее можно пытаться исправить.

Идея TD в том, чтобы смотреть на разницу между нашей прошлой оценкой и уточненной и использовать эту информацию

Temporal - так-как есть оценка сейчас и есть уточненная оценка через один шаг

В теории можно смотреть через n-шагов

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-31.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-32.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-33.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-34.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Проблема может быть в том, что робот впадет в какой-то локально оптимальный оптимум (аля шаг вперед и упасть и получить +1) и уже оттуда не выберется, так-как ему всегда выгодно брать оптмальной действие.

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-36.png" alt="Drawing" width="500"> 


<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-37.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-39.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Иногда нужно выполять случайные, непривычные действия, чтобы познавать мир

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-40.png" alt="Drawing" width="500"> 


<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-44.png" alt="Drawing" /> 

Чему будет равняться Q функция в клетках?

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-48.png" alt="Drawing" /> 

Наша Q функция не учитывает тот факт, что агент иногда делает случайные действия. Это из-за max (так-как агент всегда смотрит на лучший исход).

Алгоритм SARSA считает более реалистичные оценки. Так-как учитывает прыжки робота в лаву, то есть считает оценку по $\epsilon -greedy$ политике.

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-50.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-52.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-53.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-54.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

SARSA сходится к оптимальной политике по вероятности , но Q может сойтись к неоптималной, если есть $\epsilon$-greedy
SARSA выигрывает во время обучения, так-как учитывает нашу политику обучения

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-59.png" alt="Drawing" width="500"> 


Off-policy подход позволяет учиться на исторических данных или на записанных заранее действиях человека. On-policy - *только на собственных действиях агента*. Это довольно важное деление, так как обучение на собственных действиях просто недоступно для многих задач. Плохо ли автомобилю с автопилотом въехать в стену? Надо попробовать, так будет работать on-policy метод. А лучше взять понимание, что это плохо из готовых данных, синтетических, например и усвоить этот опыт. Даже если мы используем симуляцию среды, а не реальную среду, то количество эпизодов симуляции обычно порядка сотен тысяч и их симуляция съедает много времени. Каждый раз проходить переобучение с нуля очень неудобно, а возможность импортировать накопленный опыт симуляции среды сильно экономит ресурсы. Однако off-policy методы можно применить не всегда.

## Аппроксимация Q-функции (Q-таблицы)

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-6.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-8.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-9.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-10.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-11.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-12.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

Очень важно, что мы предполагаем $Q(S_{t+1}, A_t)$ как const, хотя на самом деле обучение нашей функции в теории влияет и на него. Поэтому метод и называется semi-gradient. Не забываем в реализации про detach() и no_grad()

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/coursera-4-24.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/coursera-4-25.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-14.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-15.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>


$\epsilon -greedy$ нужна для исследования среды

## Проблемы DQN и их решение

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-16.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-17.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

## Experience replay

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-18.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-3-62.png" alt="Drawing" /> </td>
</tr></table>

В большинстве окружений информация, получаемая агентом распределена не независимо. Т.е. последовательные наблюдения агента сильно коррелированы между собой (что понятно из интуитивных соображений, т.к. большинство окружений, в которых применяется RL, предполагают, что все изменения в них последовательны). Корреляция примеров ухудшает сходимость стохастического градиентного спуска. Таким образом нам нужен способ, который позволяет улучшить распределение примеров для обучения (устранить или снизить корреляцию между ними). Обычно используется метод **проигрывания опыта (experience replay)**. Суть этого метода в том, что мы сохраняем некоторое количество примеров (состояние, действия, вознаграждение) в специальном буфере и для обучения выбираем случайные мини-батчи из этого буфера.

Так же **experience replay** позволяет агенту эффективнее использовать свой прошлый опыт.


Стоит отметить, что нашим состоянием в DQN является картинка и хранить много картинок это не дешево

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-23.png" alt="Drawing" width="500"> 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-24.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-25.png" alt="Drawing" /> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-26.png" alt="Drawing" /> 

Одной из проблем Q-Networks является неустойчивость. Часто разность ожидаемых вознаграждений для различных действий близка и поскольку выбор действия производится с помощью argmax, то выброс в данных может привести к тому, что выбираемое действие изменится. Для того, чтобы повысить стабильность используется техника **Target Q-Network**. Суть в том, что мы замораживаем веса нашей сети на фиксированное число шагов и затем используем её для вычисления функции ошибки и обучения второй сети. Периодически копируем из веса рабочей сети в Target Q-Network.

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-28.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-29.png" alt="Drawing" /> </td>
</tr></table>

На слайде справа показаны все компоненты с помощю которых была реализована в 2013 первая DQN, играющая в Atari 

## Пример c CartPole


### Описание CartPole

CartPole это одна из классических задач RL, которая была введене Barto, Sutton, Anderson в 1983 году. В ней шест (Pole) прикреплен к тележке (Cart), которая движется по дорожке влево/вправо без трения. Система управляется приложением к тележке силы +1 или -1. Маятник запускается вертикально, и цель состоит в том, чтобы не допустить его падения. Эпизод заканчивается, когда штанга отклоняется от вертикали более чем на 15 градусов или тележка перемещается более чем на 2.4 единицы от центра. Награда +1 дается за каждый такт времени. Очевидно, что если  эпизод закончится, то мы перестанем получать награды и наш выигрыш составит то количество очков которое мы успели набрать.

Каждое состояние описываетcя 4 значениями:


|N|Action|min|max|
|------|------|------|------|
|0|Cart Position|-2.4|2.4|
|1|Cart Velocity|-Inf|Inf|
|2|Pole Angle|-41.8°| 41.8°|
|3|Pole VelocityAt Tip|-Inf|Inf|


Ко всем наблюдениям добавляется случайный шум из равномерное распределения U(-0.05,+0.05)

### Задача

Приведенный ниже код можно использовать без модификации для других env(игр). Единстенное требование, что пространство состояний должно быть единым вектором, действия должны быть дискретными.

CartPole - самый простой. На ее решение должно уйти несколько минут.

Для LunarLander может потребоваться 1-2 часа, чтобы набрать 200 баллов (хороший результат) на Colab, и прогресс обучения не выглядит информативным.

In [ ]:
ENV_NAME = 'CartPole-v1'

def make_env(seed=None):
    # some envs are wrapped with a time limit wrapper by default
    env = gym.make(ENV_NAME).unwrapped
    if seed is not None:
        env.seed(seed)
    return env

In [ ]:
env = make_env()
env.reset()
plt.imshow(env.render("rgb_array"))
state_shape, n_actions = env.observation_space.shape, env.action_space.n

### Building a network

Теперь нам нужно построить нейронную сеть, которая cможет отображать состояние (state) в набор q-значений для этого состония.
Модель не обязательно должна быть большой. Возможно, будет достаточно 1-2 скрытых слоев с <200 нейронами и функцией активации ReLU. Batch normalization и dropout могут здесь все испортить.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# those who have a GPU but feel unfair to use it can uncomment:
# device = torch.device('cpu')
device
print(state_shape)

In [ ]:
class DQNAgent(nn.Module):
    def __init__(self, state_shape, n_actions, epsilon=0):

        super().__init__()
        self.epsilon = epsilon
        self.n_actions = n_actions
        self.state_shape = state_shape
        # Define your network body here. Please make sure agent is fully contained here
        assert len(state_shape) == 1
        state_dim = state_shape[0]
        
        
        # Define NN
        ##############################################
        hidden_size = 150
        self._nn = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions),
            nn.ReLU()
        )
        ##############################################

        
    def forward(self, state_t):
        """
        takes agent's observation (tensor), returns qvalues (tensor)
        :param state_t: a batch states, shape = [batch_size, *state_dim=4]
        """
        # Use your network to compute qvalues for given state
        
        ##############################################
        qvalues = self._nn(state_t)
        ##############################################

        assert qvalues.requires_grad, "qvalues must be a torch tensor with grad"
        assert (
            len(qvalues.shape) == 2 and 
            qvalues.shape[0] == state_t.shape[0] and 
            qvalues.shape[1] == n_actions
        )

        return qvalues

    def get_qvalues(self, states):
        """
        like forward, but works on numpy arrays, not tensors
        """
        model_device = next(self.parameters()).device
        states = torch.tensor(states, device=model_device, dtype=torch.float32)
        qvalues = self.forward(states)
        return qvalues.data.cpu().numpy()

    def sample_actions(self, qvalues):
        """pick actions given qvalues. Uses epsilon-greedy exploration strategy. """
        epsilon = self.epsilon
        batch_size, n_actions = qvalues.shape

        random_actions = np.random.choice(n_actions, size=batch_size)
        best_actions = qvalues.argmax(axis=-1)

        should_explore = np.random.choice(
            [0, 1], batch_size, p=[1-epsilon, epsilon])
        return np.where(should_explore, random_actions, best_actions)

In [ ]:
agent = DQNAgent(state_shape, n_actions, epsilon=0.5).to(device)

In [ ]:
def evaluate(env, agent, n_games=1, greedy=False, t_max=10000):
    """ Plays n_games full games. If greedy, picks actions as argmax(qvalues). Returns mean reward. """
    rewards = []
    for _ in range(n_games):
        s = env.reset()
        reward = 0
        for _ in range(t_max):
            qvalues = agent.get_qvalues([s])
            action = qvalues.argmax(axis=-1)[0] if greedy else agent.sample_actions(qvalues)[0]
            s, r, done, _ = env.step(action)
            reward += r
            if done:
                break

        rewards.append(reward)
    return np.mean(rewards)

### Experience Replay Buffer and Target Networks

#### Интерфейс взаимодействия с replay buffer очень прост:
* `exp_replay.add(obs, act, rw, next_obs, done)` - saves (s,a,r,s',done) tuple into the buffer
* `exp_replay.sample(batch_size)` - returns observations, actions, rewards, next_observations and is_done for `batch_size` random samples.
* `len(exp_replay)` - returns number of elements stored in replay buffer.

In [ ]:
from replay_buffer import ReplayBuffer

exp_replay = ReplayBuffer(2000)


target_network = DQNAgent(agent.state_shape, agent.n_actions, epsilon=0.5).to(device)
# This is how you can load weights from agent into target network
target_network.load_state_dict(agent.state_dict())

### TD-Loss

Вычислим Q-learning TD error:

$$ L = { 1 \over N} \sum_i [ Q_{\theta}(s,a) - Q_{reference}(s,a) ] ^2 $$

где Q-reference определен следующим образом:

$$ Q_{reference}(s,a) = r(s,a) + \gamma \cdot max_{a'} Q_{target}(s', a') $$

Где
* $Q_{target}(s',a')$ это Q-value для следующего состояния (next state) и следующего действия (next action), которое предсказывается __target_network__
* $s, a, r, s'$ - текущее состояние (current state), действие (action), награда (reward) и следующее состояние (next state) соответсвенно
* $\gamma$ - discount factor

In [ ]:
def compute_td_loss(states, actions, rewards, next_states, is_done,
                    agent, target_network,
                    gamma=0.99,
                    check_shapes=False,
                    device=device):
    """ Compute td loss using torch operations only. Use the formulae above. """
    states = torch.tensor(states, device=device, dtype=torch.float32)    # shape: [batch_size, *state_shape]
    actions = torch.tensor(actions, device=device, dtype=torch.int64)    # shape: [batch_size]
    rewards = torch.tensor(rewards, device=device, dtype=torch.float32)  # shape: [batch_size]
    # shape: [batch_size, *state_shape]
    next_states = torch.tensor(next_states, device=device, dtype=torch.float)
    is_done = torch.tensor(
        is_done.astype('float32'),
        device=device,
        dtype=torch.float32,
    )  # shape: [batch_size]
    is_not_done = 1 - is_done

    # get q-values for all actions in current states
    predicted_qvalues = agent(states)  # shape: [batch_size, n_actions]

    # compute q-values for all actions in next states
    # with torch.no_grad():
    predicted_next_qvalues = target_network(next_states)  # shape: [batch_size, n_actions]
    
    # select q-values for chosen actions
    predicted_qvalues_for_actions = predicted_qvalues[range(len(actions)), actions]  # shape: [batch_size]

    # compute V*(next_states) using predicted next q-values
    ##############################################
    next_state_values = predicted_next_qvalues.max(axis=-1)[0]
    ##############################################

    assert next_state_values.dim() == 1 and next_state_values.shape[0] == states.shape[0], \
        "must predict one value per state"

    # compute "target q-values" for loss - it's what's inside square parentheses in the above formula.
    # at the last state use the simplified formula: Q(s,a) = r(s,a) since s' doesn't exist
    # you can multiply next state values by is_not_done to achieve this.
    ###############################################
    target_qvalues_for_actions = rewards + gamma * next_state_values * is_not_done
    ##############################################

    # mean squared error loss to minimize
    loss = torch.mean((predicted_qvalues_for_actions - target_qvalues_for_actions.detach()) ** 2)

    if check_shapes:
        assert predicted_next_qvalues.data.dim() == 2, \
            "make sure you predicted q-values for all actions in next state"
        assert next_state_values.data.dim() == 1, \
            "make sure you computed V(s') as maximum over just the actions axis and not all axes"
        assert target_qvalues_for_actions.data.dim() == 1, \
            "there's something wrong with target q-values, they must be a vector"

    return loss

### Main loop

In [ ]:
from tqdm import trange
from IPython.display import clear_output
import matplotlib.pyplot as plt
import time

In [ ]:
# your favourite random seed
seed = 2
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

env = make_env(seed)
state_dim = env.observation_space.shape
n_actions = env.action_space.n
state = env.reset()

agent = DQNAgent(state_dim, n_actions, epsilon=1).to(device)
target_network = DQNAgent(state_dim, n_actions, epsilon=1).to(device)
target_network.load_state_dict(agent.state_dict())

In [ ]:
def play_and_record(initial_state, agent, env, exp_replay, n_steps=1):
    """
    Play the game for exactly n_steps, record every (s,a,r,s', done) to replay buffer. 
    Whenever game ends, add record with done=True and reset the game.
    It is guaranteed that env has done=False when passed to this function.

    PLEASE DO NOT RESET ENV UNLESS IT IS "DONE"

    :returns: return sum of rewards over time and the state in which the env stays
    
    hint: use agent.sample.actions
    """
    s = initial_state
    sum_rewards = 0

    # Play the game for n_steps as per instructions above
    # <YOUR CODE>
    for _ in range(n_steps):
        qvalues = agent.get_qvalues([s])
        
        action = agent.sample_actions(qvalues)[0]
        # action = action.argmax(axis=-1)[0]
        state, reward, done, _ = env.step(action)
        sum_rewards += reward
        
        exp_replay.add(s, action, reward, state, done)
        
        if done:
            state = env.reset()
        
        s = state
        
    

    return sum_rewards, s

In [ ]:
REPLAY_BUFFER_SIZE = 10**4

exp_replay = ReplayBuffer(REPLAY_BUFFER_SIZE)
for i in range(100):
    if not utils.is_enough_ram(min_available_gb=0.1):
        print("""
            Less than 100 Mb RAM available. 
            Make sure the buffer size in not too huge.
            Also check, maybe other processes consume RAM heavily.
            """
             )
        break
    play_and_record(state, agent, env, exp_replay, n_steps=10**2)
    if len(exp_replay) == REPLAY_BUFFER_SIZE:
        break
print(len(exp_replay))

In [ ]:
timesteps_per_epoch = 1
batch_size = 32
total_steps = 4 * 10**4
decay_steps = 1 * 10**4

opt = torch.optim.Adam(agent.parameters(), lr=1e-4)

init_epsilon = 1
final_epsilon = 0.1

loss_freq = 20
refresh_target_network_freq = 100
eval_freq = 1000

max_grad_norm = 5000

mean_rw_history = []
td_loss_history = []
grad_norm_history = []
initial_state_v_history = []
step = 0

def wait_for_keyboard_interrupt():
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        pass

In [ ]:
state = env.reset()
with trange(step, total_steps + 1) as progress_bar:
    for step in progress_bar:
        if not utils.is_enough_ram():
            print('less that 100 Mb RAM available, freezing')
            print('make sure everything is ok and use KeyboardInterrupt to continue')
            wait_for_keyboard_interrupt()

        agent.epsilon = utils.linear_decay(init_epsilon, final_epsilon, step, decay_steps)

        # play
        _, state = play_and_record(state, agent, env, exp_replay, timesteps_per_epoch)

        # train
        # <YOUR CODE: sample batch_size of data from experience replay>
        s,a,r,next_s, is_done = exp_replay.sample(batch_size)
        # loss = <YOUR CODE: compute TD loss>
        loss = compute_td_loss(s, a, r, next_s, is_done, agent, target_network)

        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
        opt.step()
        opt.zero_grad()

        if step % loss_freq == 0:
            td_loss_history.append(loss.data.cpu().item())
            grad_norm_history.append(grad_norm)

        if step % refresh_target_network_freq == 0:
            # Load agent weights into target_network
            # <YOUR CODE>
            target_network.load_state_dict(agent.state_dict())

        if step % eval_freq == 0:
            mean_rw_history.append(evaluate(
                make_env(seed=step), agent, n_games=3, greedy=True, t_max=1000)
            )
            initial_state_q_values = agent.get_qvalues(
                [make_env(seed=step).reset()]
            )
            initial_state_v_history.append(np.max(initial_state_q_values))

            clear_output(True)
            print("buffer size = %i, epsilon = %.5f" %
                (len(exp_replay), agent.epsilon))

            plt.figure(figsize=[16, 9])

            plt.subplot(2, 2, 1)
            plt.title("Mean reward per episode")
            plt.plot(mean_rw_history)
            plt.grid()

            assert not np.isnan(td_loss_history[-1])
            plt.subplot(2, 2, 2)
            plt.title("TD loss history (smoothened)")
            plt.plot(utils.smoothen(td_loss_history))
            plt.grid()

            plt.subplot(2, 2, 3)
            plt.title("Initial state V")
            plt.plot(initial_state_v_history)
            plt.grid()

            plt.subplot(2, 2, 4)
            plt.title("Grad norm history (smoothened)")
            plt.plot(utils.smoothen(grad_norm_history))
            plt.grid()

            plt.show()

In [ ]:
final_score = evaluate(
  make_env(),
  agent, n_games=30, greedy=True, t_max=1000
)
print('final score:', final_score)
assert final_score > 300, 'not good enough for DQN'
print('Well done')

## Дальнейшие идеи для улучшения

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-30.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-31.png" alt="Drawing" /> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-32.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-33.png" alt="Drawing" /> </td>
</tr></table>

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-34.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-35.png" alt="Drawing" /> </td>
</tr></table>

То есть, допустим, что Q(s,a) распределен случайно. То есть, выбор action никак не влияет. Но так-как мы берем max по Q, то будет казаться, что мы можем получать большие значения

## Double DQN


<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-36.png" alt="Drawing" width="500">

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-37.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-38.png" alt="Drawing" /> </td>
</tr></table>


<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-39.png" alt="Drawing" width="500"> 

<table><tr>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-48.png" alt="Drawing" /> </td>
<td> <img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-49.png" alt="Drawing" /> </td>
</tr></table>

<img src="https://edunet.kea.su/repo/src/L15_RL/img/yds-4-50.png" alt="Drawing" width="500">

# Другие улучшения DQN

## Prioritized Experience Replay

Минибатчи из памяти выбираются не с равномерным распределением, а добавляем туда больше примеров, в которых предсказанные значения Q сильнее всего отличаются от корректных. Т.е. примеры с максимальным **TD error** получают максимальный приоритет.

## Dueling networks

Основная идея в том, что мы разделяем нашу сеть на две головы, одна из которых предсказывает абсолютное значение состояния \\( V(S) \\), а вторая - относительное преимущество одний действий над другими \\( A(s, a) = Q(s, a) - V(s) \\). Это преимущество называется advantage. Далее из этих двух значений мы собираем нашу Q-функцию, как \\( Q(s,a) = V(s) + A(a) \\)


## Multi-step learning/n-step learning

Основная идея в том, чтобы считать функцию ценности не по двум соседним примерам, а сразу по n. Это позволяет сети лучше запоминать длинные последовательности действий.

## Distributional RL

Детерминированное значение Q заменяется случайным распределением Z с некоторыми параметрами, которые определяются в ходе обучения.

# Rainbow

Rainbow - набор перечисленных выше твиков. 

# R2D2

Использование RNN сетей для experience replay

## Резюме по типам алгоритмов RL

### Model-free / Model-based
Model-free не строит модель окружения или функции награды. Это проще. Model-based алгоритм пытается предсказывать, каким будет следующее состояние окружения или вознаграждение. Это позволяет мыслить на несколько шагов вперёд. Например, совершенно не больно прыгать с крыши. Больно потом, когда разбиваешься о землю. Модель среды и награды позволяет принимать решения не прыгать с крыши, хотя и model-free подход позволяет это понять, хоть и более сложно и грубо.
### Value-based / policy-based
Policy-based  методы оптимизируют напрямую функцию принятия решения агента. Стратегия (policy) обычно представлена распределением вероятности доступных действий. Value-based метод оптимизирует оценку вознаграждения для всех действий и выбирает выбирает то действие, по которому прогнозируется большее значение. Методы, основанные на Policy Gradients лучше работают при большой размерности пространства действий, а Value-based методы, такие, как Deep Q-Learning требуют меньшего количества повторений для сходимости при малой размерности.
### On-Policy / Off-Policy
Off-policy подход позволяет учиться на исторических данных или на записанных заранее действиях человека. On-policy - только на собственных действиях агента. Это довольно важное деление, так как обучение на собственных действиях просто недоступно для многих задач. Плохо ли автомобилю с автопилотом въехать в стену? Надо попробовать, так будет работать on-policy метод. А лучше взять понимание, что это плохо из готовых данных, синтетических, например и усвоить этот опыт. Даже если мы используем симуляцию среды, а не реальную среду, то количество эпизодов симуляции обычно порядка сотен тысяч и их симуляция съедает много времени. Каждый раз проходить переобучение с нуля очень неудобно, а возможность импортировать накопленный опыт симуляции среды сильно экономит ресурсы. Однако off-policy методы можно применить не всегда.
### Deterministic Policy / Stochastic Policy
В зависимости от среды, наша стратегия может быть либо детерминированной - выбираем сразу определённое действие с помощью argmax, либо стохастической, когда мы окончательное решение принимается с помощью генератора случайных чисел на основе распределения вероятности, выданного сетью.